In [12]:
import nest_asyncio

nest_asyncio.apply()


In [13]:
# grab the content from the web
import requests
from bs4 import BeautifulSoup
import pandas as pd
import json

In [16]:
tickers = "https://sec.gov/files/company_tickers.json"
response = requests.get(tickers, headers={"User-Agent": "secretemail@gmail.com"})
data = response.json()

# print(data['0'])
cik = data["0"]["cik_str"]

company = pd.DataFrame.from_dict(data, orient="index")

# print(company)


# add leading 0s
company["cik_str"] = company["cik_str"].astype(str).str.zfill(10)

# since we're reading from a dataframe, we need to use the [0:1] to get the first row
cik = company[0:1].cik_str[0]

# print(cik)
filings = requests.get(
    f"https://data.sec.gov/submissions/CIK{cik}.json",
    headers={"User-Agent": "secretemail@gmail.com"},
)

# print(filings.json())

all_filings = pd.DataFrame.from_dict(filings.json()["filings"]["recent"])

# print(all_filings.columns)

all_filings[["accessionNumber", "reportDate", "form"]].where(
    all_filings["form"] == ("10-Q")
).dropna().where(all_filings["reportDate"] > "2022-01-01").dropna().sort_values(
    by="reportDate", ascending=False
)
# when accessing, limit the reportDate to the last 4 quarters

# all_filings.iloc


,accessionNumber,reportDate,form
7,0000320193-23-000064,2023-04-01,10-Q
46,0000320193-23-000006,2022-12-31,10-Q
79,0000320193-22-000070,2022-06-25,10-Q
86,0000320193-22-000059,2022-03-26,10-Q
119,0000320193-22-000007,2021-12-25,10-Q
155,0000320193-21-000065,2021-06-26,10-Q
159,0000320193-21-000056,2021-03-27,10-Q
191,0000320193-21-000010,2020-12-26,10-Q
221,0000320193-20-000062,2020-06-27,10-Q
228,0000320193-20-000052,2020-03-28,10-Q


In [27]:
# company facts
companyFacts = requests.get(
    f"https://data.sec.gov/api/xbrl/companyfacts/CIK{cik}.json",
    headers={"User-Agent": "secretemail@gmail.com"},
)
print(cik)
# print(companyFacts.json().keys())
# print(companyFacts.json()['facts'])
# print(companyFacts.json()['facts'].keys())
# print(companyFacts.json()['facts']['dei'])

# dei - EntityCommonStockSharesOutstanding and EntityPublicFloat
# us-gaap

# print(companyFacts.json()['facts']['us-gaap'])
# print(companyFacts.json()['facts']['us-gaap'].keys())

# filing metadata
# company_facts = pd.DataFrame.from_dict(companyFacts.json()['facts'])

# how do I access the 10-Q documents
companyConcept = requests.get(
    f"https://data.sec.gov/api/xbrl/companyconcept/CIK{cik}/us-gaap/Assets.json",
    headers={"User-Agent": "secretemail@gmail.com"},
)

print(companyConcept.json().keys())
print(companyConcept.json()["units"].keys())
print(companyConcept.json()["units"]["USD"][0])

# parse from single filing
print(companyConcept.json()["units"]["USD"][0]["val"])

assetsData = pd.DataFrame.from_dict(companyConcept.json()["units"]["USD"])
assetsData.columns
assetsData.form

# get assets from 10Q forms and reset index
assetsData = assetsData[assetsData.form == "10-Q"].reset_index(drop=True)
assetsData


0000320193
dict_keys(['cik', 'taxonomy', 'tag', 'label', 'description', 'entityName', 'units'])
dict_keys(['USD'])
{'end': '2008-09-27', 'val': 39572000000, 'accn': '0001193125-09-153165', 'fy': 2009, 'fp': 'Q3', 'form': '10-Q', 'filed': '2009-07-22'}
39572000000


,end,val,accn,fy,fp,form,filed,frame
0,2008-09-27,39572000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,NaN
1,2009-06-27,48140000000,0001193125-09-153165,2009,Q3,10-Q,2009-07-22,CY2009Q2I
2,2009-09-26,47501000000,0001193125-10-012085,2010,Q1,10-Q,2010-01-25,NaN
3,2009-09-26,47501000000,0001193125-10-088957,2010,Q2,10-Q,2010-04-21,NaN
4,2009-09-26,47501000000,0001193125-10-162840,2010,Q3,10-Q,2010-07-21,NaN
...,...,...,...,...,...,...,...,...
79,2022-06-25,336309000000,0000320193-22-000070,2022,Q3,10-Q,2022-07-29,CY2022Q2I
80,2022-09-24,352755000000,0000320193-23-000006,2023,Q1,10-Q,2023-02-03,NaN
81,2022-09-24,352755000000,0000320193-23-000064,2023,Q2,10-Q,2023-05-05,CY2022Q3I
82,2022-12-31,346747000000,0000320193-23-000006,2023,Q1,10-Q,2023-02-03,CY2022Q4I


In [30]:
recent_10qs = assetsData.where(assetsData.filed > "2022-01-01").dropna()
recent_10qs

companyFrames = requests.get(
    f"https://data.sec.gov/api/xbrl/frames/us-gaap/AccountsPayableCurrent/USD/{frame}.json"
)


,end,val,accn,fy,fp,form,filed,frame
59,2018-12-29,3.737190e+11,0000320193-19-000010,2019.0,Q1,10-Q,2019-01-30,CY2018Q4I
60,2019-03-30,3.419980e+11,0000320193-19-000066,2019.0,Q2,10-Q,2019-05-01,CY2019Q1I
61,2019-06-29,3.222390e+11,0000320193-19-000076,2019.0,Q3,10-Q,2019-07-31,CY2019Q2I
65,2019-12-28,3.406180e+11,0000320193-20-000010,2020.0,Q1,10-Q,2020-01-29,CY2019Q4I
66,2020-03-28,3.204000e+11,0000320193-20-000052,2020.0,Q2,10-Q,2020-05-01,CY2020Q1I
67,2020-06-27,3.173440e+11,0000320193-20-000062,2020.0,Q3,10-Q,2020-07-31,CY2020Q2I
71,2020-12-26,3.540540e+11,0000320193-21-000010,2021.0,Q1,10-Q,2021-01-28,CY2020Q4I
72,2021-03-27,3.371580e+11,0000320193-21-000056,2021.0,Q2,10-Q,2021-04-29,CY2021Q1I
73,2021-06-26,3.298400e+11,0000320193-21-000065,2021.0,Q3,10-Q,2021-07-28,CY2021Q2I
77,2021-12-25,3.811910e+11,0000320193-22-000007,2022.0,Q1,10-Q,2022-01-28,CY2021Q4I
